In [0]:
# %% [code]

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.utils import shuffle
train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')

train_and_test=[train,test]

for dataset in train_and_test:
    dataset['Title']=dataset.Name.str.extract(' (A-Za-z]+)\.')



for dataset in train_and_test:
    
    dataset['Title'] = dataset['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer',
                                                 'Lady','Major', 'Rev', 'Sir'], 'Other')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')

train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

for dataset in train_and_test:
    dataset['Title'] = dataset['Title'].astype(str)
    

      
for data in train_and_test:
    dataset['Sex']=dataset['Sex'].astype(str)




# %% [code]
def pie_chart(feature):
    features_ratio=train[feature].value_counts(sort=False)
    features_size=features_ratio.size
    features_index=features_ratio.index
    survived=train[train['Survived']==1][feature].value_counts()
    dead=train[train['Survived']==0][feature].value_counts()
    plt.plot(aspect='auto')
    plt.pie(features_ratio,labels=features_index,autopct='%1.1f%%')
    plt.title(feature+'\'s ratio in total')
    plt.show()
    
    for i,index in enumerate(features_index):
        
        plt.subplot(1,features_size+1,i+1,aspect='equal')
        plt.pie([survived[index],dead[index]],labels=['Survived','Dead'],autopct='%1.1f%%')
        plt.title(str(index)+'\'s ratio')
    plt.show()
pie_chart('Sex')

# %% [code]
pie_chart('Pclass')

# %% [code]
def bar_chart(feature):
    survived=train[train['Survived']==1][feature].value_counts()
    dead=train[train['Survived']==0][feature].value_counts()
    df=pd.DataFrame([survived,dead])
    df.index=['Survived','Dead']
    df.plot(kind='bar',stacked=True,figsize=(10,5))
    
bar_chart("SibSp")
bar_chart("Parch")

# %% [code]
train.Embarked.value_counts(dropna=False)

# %% [code]
for dataset in train_and_test:
    dataset['Embarked']=dataset['Embarked'].fillna('S')
    dataset['Embarked']=dataset['Embarked'].astype(str)
    

# %% [code]
for dataset in train_and_test:
    dataset['Age'].fillna(dataset['Age'].mean(),inplace=True)
    dataset['Age']=dataset['Age'].astype(int)
    train['AgeBand']=pd.cut(train['Age'],5)
print(train[['AgeBand','Survived']].groupby(['AgeBand'],as_index=False).mean())
train.head()

# %% [code]
for dataset in train_and_test:
    dataset.loc[dataset['Age']<=16,'Age']=0
    dataset.loc[(dataset['Age']>16) & (dataset['Age']<=32), 'Age']=1
    dataset.loc[(dataset['Age']>32) & (dataset['Age']<=48),'Age']=2
    dataset.loc[(dataset['Age']>48) & (dataset['Age']<=64),'Age']=3
    dataset.loc[dataset['Age']>64,'Age']=4
    dataset['Age']=dataset['Age'].map({0:'Child',1:'Young',2:'Middle',3:'Prime',4:'Old'}).astype(str)
    
train.head()
    
    

# %% [code]
print(train[['Pclass','Fare']].groupby(['Pclass'],as_index=False).mean())
print("")
print(test[test["Fare"].isnull()]["Pclass"])



# %% [code]
for dataset in train_and_test:
        dataset['Fare']=dataset['Fare'].fillna(13.675)

# %% [code]
for dataset in train_and_test:
    dataset["Family"]=dataset["Parch"]+dataset["SibSp"]
    dataset['Family']=dataset['Family'].astype(int)
    

# %% [code]
features_drop=['Name','Ticket','Cabin','SibSp','Parch']
train=train.drop(features_drop,axis=1)
test=test.drop(features_drop,axis=1)
train=train.drop(['PassengerId','AgeBand'],axis=1)
print(train.head())
print(test.head())

# %% [code]
train=pd.get_dummies(train)
test=pd.get_dummies(test)

train_label=train['Survived']
train_data=train.drop('Survived',axis=1)
test_data=test.drop("PassengerId",axis=1).copy()


# %% [code]
train_data,train_label=shuffle(train_data,train_label,random_state=5)

# %% [code]
def train_and_test(model):
    model.fit(train_data,train_label)
    prediction=model.predict(test_data)
    accuracy=round(model.score(train_data,train_label)*100,2)
    print("Accuracy:",accuracy,"%")
    return prediction



# %% [code]
log_pred=train_and_test(LogisticRegression())

svm_pred=train_and_test(SVC())

knn_pred_4=train_and_test(KNeighborsClassifier(n_neighbors=4))

rf_pred=train_and_test(RandomForestClassifier(n_estimators=100))

np_pred=train_and_test(GaussianNB())

# %% [code]
submission=pd.DataFrame({"PassengerId":test["PassengerId"],"Survived":rf_pred})

submission.to_csv('submission_rf.csv',index=False)